<a href="https://colab.research.google.com/github/BarrenWardo/Py-Books/blob/main/sd_cui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone "https://github.com/BarrenWardo/CUI"

In [ ]:
%cd "/content/CUI"
!sudo apt-get install aria2 -y
!pip install -r requirements.txt
!pip install pyngrok

In [ ]:
%cd "/content/CUI/custom_nodes"
!git clone "https://github.com/ltdrdata/ComfyUI-Manager"
!git clone "https://github.com/pythongosssss/ComfyUI-Custom-Scripts"
!git clone "https://github.com/ssitu/ComfyUI_UltimateSDUpscale" --recursive
!git clone "https://github.com/civitai/civitai_comfy_nodes"

In [ ]:
#Models Download

!aria2c "https://civitai.com/api/download/models/240840?type=Model&format=SafeTensor&size=full&fp=fp16" -x16 -d "/content/CUI/models/checkpoints" --summary-interval=5 --file-allocation=none --keep-unfinished-download-result=false -c --download-result=full

#4xUltraSharp
!aria2c "https://civitai.com/api/download/models/125843?type=Model&format=PickleTensor" -d "/content/CUI/models/upscale_models" --summary-interval=5 --file-allocation=none --keep-unfinished-download-result=false -c --download-result=full
#SDXL VAE
!aria2c "https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors?download=true" -d "/content/CUI/models/vae" -o "sdxl_vae.safetensors" --summary-interval=5 --file-allocation=none --keep-unfinished-download-result=false -c --download-result=full
#1.5 VAE
!aria2c "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors?download=true" -d "/content/CUI/models/vae" -o "vae-ft-mse-840000-ema-pruned.safetensors" --summary-interval=5 --file-allocation=none --keep-unfinished-download-result=false -c --download-result=full

#TEMPLATES - Remove # and replace download-link in aria2c code line
#Checkpoints
#!aria2c "download-link" -d "/content/CUI/models/checkpoints" --summary-interval=5 --file-allocation=none --keep-unfinished-download-result=false -c --download-result=full
#ControlNet
#!aria2c "download-link" -d "/content/CUI/models/controlnet" --summary-interval=5 --file-allocation=none --keep-unfinished-download-result=false -c --download-result=full
#Embeddings / Textual Inversions
#!aria2c "download-link" -d "/content/CUI/models/embeddings" --summary-interval=5 --file-allocation=none --keep-unfinished-download-result=false -c --download-result=full
#LoRAs
#!aria2c "download-link" -d "/content/CUI/models/loras" --summary-interval=5 --file-allocation=none --keep-unfinished-download-result=false -c --download-result=full
#Upscalers
#!aria2c "download-link" -d "/content/CUI/models/upscale_models" --summary-interval=5 --file-allocation=none --keep-unfinished-download-result=false -c --download-result=full
#VAE
#!aria2c "download-link" -d "/content/CUI/models/vae" --summary-interval=5 --file-allocation=none --keep-unfinished-download-result=false -c --download-result=full

In [ ]:
#NGROK

%cd "/content/CUI"

from pyngrok import ngrok

ngrok.set_auth_token("your-ngrok-auth-token")

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    print("\nComfyUI finished loading, trying to launch ngrok (if it gets stuck here ngrok is having issues)\n")

    public_url = ngrok.connect(port)
    print(f"Ngrok tunnel is live at: {public_url}")

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py \
    --dont-print-server